In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#parameters
learning_rate=0.001
training_epochs=15
batch_size=128
SUMMARY_DIR='./mnist'

#load mnist dataset
MNIST=input_data.read_data_sets("./MINST_data", one_hot=True)

Extracting ./MINST_data\train-images-idx3-ubyte.gz
Extracting ./MINST_data\train-labels-idx1-ubyte.gz
Extracting ./MINST_data\t10k-images-idx3-ubyte.gz
Extracting ./MINST_data\t10k-labels-idx1-ubyte.gz


In [3]:
#0부터 examples수 만큼의 list
example_list = np.arange(0,MNIST.train.num_examples)

def get_train_batch(batch_size) :
    start = i * batch_size   #시작 index 저장 
    if i == 0 :
        np.random.shuffle(example_list)   #idx list를 shuffle        
    if i != batch_cnt :
            end = start + batch_size
    else :                  #마지막 mini batch의 end index 설정
            end = start + int(MNIST.train.num_examples)%batch_size
    local_id = example_list[start:end]
    x = [MNIST.train._images[j] for j in local_id]
    y = [MNIST.train._labels[k] for k in local_id]
        
    return x,y

In [4]:
with tf.name_scope('input') as scope:
    X=tf.placeholder(tf.float32, [None, 784], name='image')
    y=tf.placeholder(tf.float32, [None, 10], name='label')

with tf.variable_scope('layer1') as scope:
    W1=tf.get_variable("W", shape=[784,512], initializer=tf.contrib.layers.xavier_initializer())
    b1=tf.Variable(tf.random_normal([512]))
    L1=tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
    
    tf.summary.histogram("X", X)
    tf.summary.histogram("weights", W1)
    tf.summary.histogram("bias", b1)
    tf.summary.histogram("layer", L1)

with tf.variable_scope('layer2') as scope:
    W2=tf.get_variable("W", shape=[512,1024], initializer=tf.contrib.layers.xavier_initializer())
    b2=tf.Variable(tf.random_normal([1024]))
    L2=tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
    
    tf.summary.histogram("X", X)
    tf.summary.histogram("weights", W2)
    tf.summary.histogram("bias", b2)
    tf.summary.histogram("layer", L2)
    
with tf.variable_scope('layer3') as scope:
    W3=tf.get_variable("W", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
    b3=tf.Variable(tf.random_normal([10]))
    y_=tf.add(tf.matmul(L2, W3), b3)
    
    tf.summary.histogram("weights", W3)
    tf.summary.histogram("bias", b3)
    tf.summary.histogram("logits", y_)

In [5]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
tf.summary.scalar("loss", loss)

#merge all summaries
summary=tf.summary.merge_all()

In [6]:
global_step=0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    for epoch in range(training_epochs):
        total_batch=int(MNIST.train.num_examples/batch_size)+1
        batch_cnt = total_batch - 1
        avg_loss=0
        
        for i in range(total_batch):
            batch_xs, batch_ys = get_train_batch(batch_size)   
        # batch_xs, batch_ys = MNIST.train.next_batch(batch_size)
            feed_dict={X:batch_xs, y:batch_ys}
            s, l, _=sess.run([summary, loss, optimizer], feed_dict=feed_dict)
            writer.add_summary(s, global_step=global_step)
            global_step+=1
            avg_loss+=l
        print('Epoch:','%02d'%(epoch+1),'loss=','{:.6f}'.format(avg_loss/total_batch))
    
    correct_prediction=tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc=sess.run(accuracy, feed_dict={X:MNIST.test.images, y:MNIST.test.labels})
    print('Test accuracy:', acc)

Epoch: 01 loss= 2.098340
Epoch: 02 loss= 1.714840
Epoch: 03 loss= 1.407824
Epoch: 04 loss= 1.164516
Epoch: 05 loss= 0.984222
Epoch: 06 loss= 0.855008
Epoch: 07 loss= 0.762069
Epoch: 08 loss= 0.693061
Epoch: 09 loss= 0.640902
Epoch: 10 loss= 0.600035
Epoch: 11 loss= 0.567196
Epoch: 12 loss= 0.540224
Epoch: 13 loss= 0.517976
Epoch: 14 loss= 0.498961
Epoch: 15 loss= 0.482675
Test accuracy: 0.884
